Khi chúng ta đã chọn một kiến ​​​​trúc và đặt siêu tham số của mình, chúng tôi sẽ tiến hành vòng lặp huấn luyện, trong đó mục tiêu của chúng ta là tìm các giá trị tham số giúp giảm thiểu hàm mất mát của chúng ta. Sau khi đào tạo, chúng ta sẽ cần các tham số này để đưa ra dự đoán trong tương lai. Ngoài ra, đôi khi chúng ta muốn trích xuất các tham số để sử dụng lại chúng trong một số ngữ cảnh khác, để lưu mô hình của chúng ta vào đĩa để nó có thể được thực thi trong phần mềm khác hoặc để kiểm tra với hy vọng đạt được hiểu biết khoa học.

Hầu hết thời gian, chúng ta sẽ có thể bỏ qua các chi tiết cơ bản về cách khai báo và thao tác các tham số, dựa vào các khung học sâu để thực hiện công việc nặng nhọc. Tuy nhiên, khi chúng ta rời xa các kiến ​​trúc xếp chồng lên nhau với các lớp tiêu chuẩn, đôi khi chúng ta sẽ cần phải đi sâu vào việc khai báo và thao tác các tham số. Trong phần này, chúng ta bao gồm những điều sau đây:
- Truy cập các tham số để gỡ lỗi, chẩn đoán và trực quan hóa.
- Chia sẻ tham số giữa các thành phần mô hình khác nhau.

In [2]:
import torch
from torch import nn

Chúng ta bắt đầu bằng cách tập trung vào MLP với một lớp ẩn.

In [3]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

# 6.2.1. Truy cập thông số

Hãy bắt đầu với cách truy cập các tham số từ các mô hình mà bạn đã biết.

Khi một mô hình được xác định thông qua class Sequential, trước tiên chúng ta có thể truy cập bất kỳ lớp nào bằng cách lập chỉ mục vào mô hình như thể đó là một danh sách. Các tham số của mỗi lớp được định vị thuận tiện trong thuộc tính của nó.

Chúng ta có thể kiểm tra các tham số của lớp được kết nối đầy đủ thứ hai như sau.

In [7]:
net[2].state_dict()

tensor([[ 0.3220,  0.1359, -0.1583,  0.2529,  0.0986,  0.1812, -0.3453, -0.2230]])

Chúng ta có thể thấy rằng lớp được kết nối đầy đủ này chứa hai tham số, tương ứng với trọng số và độ lệch của lớp đó.

## 6.2.1.1. Tham số được chọn

Lưu ý rằng mỗi tham số được biểu diễn dưới dạng một thể hiện của lớp tham số. Để làm bất cứ điều gì hữu ích với các tham số, trước tiên chúng ta cần truy cập các giá trị số cơ bản. Có nhiều hướng khác nhau để làm điều đó. Một số đơn giản hơn trong khi những cái khác tổng quát hơn. Đoạn mã sau trích xuất độ lệch từ lớp mạng thần kinh thứ hai, lớp này trả về một thể hiện của lớp tham số và tiếp tục truy cập giá trị của tham số đó.

In [8]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.2637]))

Tham số là các đối tượng phức tạp, chứa các giá trị, độ dốc và thông tin bổ sung. Đó là lý do tại sao chúng ta cần yêu cầu giá trị một cách rõ ràng.

Ngoài giá trị, mỗi tham số còn cho phép chúng ta truy cập vào gradient. Bởi vì chúng ta chưa gọi lan truyền ngược cho mạng này, nên nó đang ở trạng thái ban đầu.

In [9]:
net[2].weight.grad == None

True

## 6.2.1.2. Tất cả các tham số

Khi chúng ta cần thực hiện các thao tác trên tất cả các tham số, việc truy cập từng tham số có thể trở nên tẻ nhạt. Tình huống có thể trở nên đặc biệt khó sử dụng khi chúng ta làm việc với các mô-đun phức tạp hơn (ví dụ: các mô-đun lồng nhau), vì chúng ta sẽ cần lặp lại toàn bộ cây để trích xuất các tham số của từng mô-đun con. Dưới đây chúng tôi chứng minh việc truy cập các tham số của tất cả các lớp.

In [10]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

In [11]:
[(name, param.data) for name, param in net.named_parameters()]

[('0.weight', tensor([[-0.3899, -0.1978, -0.2126,  0.2613],
          [ 0.4009,  0.0756, -0.3550,  0.3520],
          [-0.1100,  0.4854,  0.0811,  0.1440],
          [-0.0711, -0.3631,  0.4948, -0.1113],
          [ 0.1633,  0.1130, -0.1368,  0.4097],
          [-0.2619,  0.1581, -0.4891, -0.4490],
          [-0.2164, -0.1625,  0.3084,  0.2617],
          [ 0.4812, -0.0942,  0.2676,  0.3508]])),
 ('0.bias',
  tensor([ 0.3032, -0.1512, -0.2374, -0.4392,  0.0437,  0.4895, -0.2958,  0.4790])),
 ('2.weight',
  tensor([[ 0.3220,  0.1359, -0.1583,  0.2529,  0.0986,  0.1812, -0.3453, -0.2230]])),
 ('2.bias', tensor([0.2637]))]

# 6.2.2. Tham số ràng buộc

Thông thường, chúng ta muốn chia sẻ các tham số trên nhiều lớp. Hãy xem làm thế nào để làm điều này một cách thanh lịch. Trong phần tiếp theo, chúng ta phân bổ một lớp được kết nối đầy đủ và sau đó sử dụng các tham số cụ thể của nó để đặt các tham số của lớp khác. Ở đây chúng ta cần chạy lan truyền về phía trước net(X) trước khi truy cập các tham số.

In [16]:
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)

print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

"""
###Thắc mắc###
  Tạo sao vì các tham số mô hình có chứa độ dốc, 
nên độ dốc của lớp ẩn thứ hai và lớp ẩn thứ ba được cộng lại với 
nhau trong quá trình lan truyền ngược.
"""

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Ví dụ này cho thấy các tham số của lớp thứ hai và thứ ba được gắn với nhau. Chúng không chỉ bằng nhau, chúng được biểu diễn bằng cùng một tensor chính xác. Do đó, nếu chúng ta thay đổi một trong các tham số thì tham số kia cũng thay đổi theo.